In [1]:
using Graphs
using PyCall
using DataStructures
# import PyPlot; const plt = PyPlot
nx = pyimport("networkx")
pyg = pyimport("torch_geometric")
include("src/weisfeiler_lehman.jl")
include("src/mpgnn.jl")
include("src/utils.jl")
setprecision(BigFloat, 2^4)
rand(BigFloat)

0.176971

In [2]:
using DataStructures

counter([1, 1, 2, 3])

Accumulator{Int64, Int64} with 3 entries:
  2 => 1
  3 => 1
  1 => 2

## WL and MPGNN basic test

In [3]:
edge_index = [
     1 1 1 2 2 3 3 3 4 4 5 5;
     2 3 5 1 3 1 2 4 3 5 1 4
]
x₀ = [
     1, 1, 1, 1, 1
]
γ = 0.1
wl_iters, wl_coloring = WeisfeilerLehman(x₀, edge_index)
gnn_coloring = MPGNN(x₀, edge_index, γ, wl_iters)

# println(wl_coloring)
# println(gnn_coloring)
# print(maximum(wl_coloring))

5-element Vector{BigFloat}:
 1.0
 2.0
 1.0
 3.0
 3.0

![Alt Text](wl.gif)

## Graph Generation experiment

Here, we create an Erdős–Rényi random graph with **n** vertices. Edges are added between pairs of vertices with probability **p**.

In [4]:
length(γs)

LoadError: UndefVarError: γs not defined

In [90]:
setprecision(BigFloat, 2^7)
γs = sort([rand(BigFloat) for _ in 1:5])
lottery_γs = []
success_sum = []
P = [p for p in 0.1 : 0.05 : 0.4]
N = [n for n in 10 : 10 : 50]
P = [0.2]


graphs = [[erdos_renyi(N[j], P[i]) for j in 1:length(N)] for i ∈ 1:length(P)]
special_graph = []
special_γ = []

for γ ∈ γs
    success_matrix = zeros(length(P), length(N))
    for i ∈ 1:length(P)
        for j in 1:length(N)
            x₀, edge_index = construct_graph(graphs[i][j])
            iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
            coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
            success_matrix[i, j] = coloring_wl == coloring_gnn #|| maximum(coloring_wl) <= maximum(coloring_gnn)
            if coloring_wl != coloring_gnn
                println("Iters: $(iters), P: $(P[i]), N: $(N[j])")
                println(coloring_wl)
                println(coloring_gnn)
                println(maximum(coloring_wl))
                println(maximum(coloring_gnn))
                println(minimum(degree(graphs[i][j])))
                println(degree(graphs[i][j]))
                push!(special_graph, graphs[i][j])
                push!(special_γ, γ)
            end
        end
    end
    push!(success_sum, sum(success_matrix))
    if sum(success_matrix) == sum(ones(length(P), length(N)))
        println("lottery ticket gamma: $(γ)")
        push!(lottery_γs, γ) 
    end
end

ones_sum = sum(ones(length(P), length(N)))
plot(γs, [ones_sum for _ ∈ γs], label="WL", dpi=1000)
plot!(γs, success_sum, xaxis=:log, label="MPGNN")
scatter!(lottery_γs, [ones_sum for _ ∈ lottery_γs], 
        series_annotations = text.(lottery_γs, :left, 8, rotation=270), 
        label="Lottery γs")
ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPGNN, Erdős Renyi")
xlabel!("γ")
ylabel!("successful simulations")
savefig("results\\erdos_renyi.png")

lottery ticket gamma: 0.1379302762698400667800988206620836239149
lottery ticket gamma: 0.1874474138773414934018945111919734092009
lottery ticket gamma: 0.8263301317794348285415157574785159508951
lottery ticket gamma: 0.9200451885939868757375673102574925168895
lottery ticket gamma: 0.9471032657654630742418816288072444806258


"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\A Single Channel\\results\\erdos_renyi.png"

In [54]:
display(special_graph)
display(special_γ)


3-element Vector{Any}:
 {30, 74} undirected simple Int64 graph
 {30, 74} undirected simple Int64 graph
 {30, 74} undirected simple Int64 graph

3-element Vector{Any}:
 0.2377626328314992636879517077816699189473
 0.5606641616034715012765396970957810702609
 0.9433444929317972860289565010003627904553

In [64]:
# plot(special_graph[1])

In [74]:
x₀, edge_index = construct_graph(special_graph[2])
iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
coloring_gnn_cursed = MPGNN(x₀, edge_index, special_γ[3], iters)
coloring_gnn = MPGNN(x₀, edge_index, 0.1, iters)

# println(coloring_wl)
# println(coloring_gnn)
println(coloring_wl == coloring_gnn_cursed)
println(coloring_wl == coloring_gnn)

false
true


In [89]:
γs = [rand(BigFloat) for _ in 1:10]
for γ in γs
    iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
    coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
    println(coloring_wl == coloring_gnn)
end

false
false
false
true
false
true
false
true
true
true


## Bits Erdos Renyi

Hacer gráfico que muestre la media y la desviación estandar considerando todos los gammas aleatorios


Hacer un plot de bits necesarios para alcanzar un optimo dado un grfo de tamaño n

In [70]:
P = [p for p in 0.1 : 0.05 : 0.5]
N = [n for n in 50 : 10 : 100]


graphs = [[erdos_renyi(N[j], P[i]) for j in 1:length(N)] for i ∈ 1:length(P)]

γs = sort([rand(BigFloat) for _ in 1:5])

precisions = [i for i ∈ 10: 5 :100]
success_rate = []
for precision ∈ precisions
    setprecision(BigFloat, precision)
    success_sum = []
    for γ ∈ γs
        success_matrix = zeros(length(P), length(N))
        for i ∈ 1:length(P)
            for j in 1:length(N)
                x₀, edge_index = construct_graph(graphs[i][j])
                iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
                coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
                success_matrix[i, j] = coloring_wl == coloring_gnn #||  maximum(coloring_wl) <= maximum(coloring_gnn)
                if coloring_wl != coloring_gnn
                    println("P: $(P[i]), N: $(N[j])")
                    println(coloring_wl)
                    println(BigFloat.(coloring_gnn))
                    println(maximum(coloring_wl))
                    println(maximum(coloring_gnn))
                end
            end
        end
        push!(success_sum, sum(success_matrix))
        # if sum(success_matrix) == sum(ones(length(P), length(N)))
        #     println("lottery ticket gamma: $(γ)")
        #     push!(lottery_γs, γ) 
        # end
    end
    push!(success_rate, maximum(success_sum))
end
ones_sum = sum(ones(length(P), length(N)))
plot(precisions, [ones_sum for _ ∈ precisions], label="WL", xaxis=:log, dpi=1000)
plot!(precisions, success_rate, label="MPGNN")
scatter!(precisions, success_rate)
ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPGNN, Erdős Renyi")
xlabel!("bits")
ylabel!("successful simulations")
savefig("results\\bits_erdos_renyi.png")

P: 0.1, N: 50
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 3.0, 21.0, 22.0, 3.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 32.0, 32.0, 33.0, 11.0, 3.0, 34.0, 3.0, 3.0, 30.0, 31.0, 3.0, 3.0, 3.0, 3.0, 11.0, 3.0]
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 3.0, 21.0, 22.0, 3.0, 23.0, 24.0, 21.0, 25.0, 26.0, 20.0, 26.0, 27.0, 27.0, 23.0, 23.0, 23.0, 28.0, 11.0, 3.0, 29.0, 3.0, 3.0, 27.0, 27.0, 3.0, 3.0, 3.0, 3.0, 11.0, 3.0]
34.0
29.0
P: 0.1, N: 60
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 3.0, 38.0, 39.0, 38.0, 40.0, 41.0, 42.0, 26.0, 43.0, 44.0, 3.0, 42.0, 44.0, 40.0, 42.0, 26.0, 42.0, 40.0, 40.0, 26.0, 40.0, 26.0, 40.0]
BigFloat[1.0, 2.0, 3.0, 

Excessive output truncated after 524289 bytes.

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 39.0, 47.0, 48.0, 49.0, 50.0, 47.0, 38.0, 51.0, 38.0, 51.0, 51.0, 51.0, 51.0, 51.0]
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 2.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 2.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 37.0, 45.0, 46.0, 47.0, 48.0, 45.0, 36.0, 49.0, 36.0, 49.0, 49.0, 49.0, 49.0, 49.0]
51.0
49.0
P: 0.2, N: 70
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45

"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\A Single Channel\\bits_erdos_renyi.png"

Here, we create a Barabási–Albert model random graph with n vertices. It is grown by adding new vertices to an initial graph with k vertices. Each new vertex is attached with k edges to k different vertices already present in the system by preferential attachment. Initial graphs are undirected and consist of isolated vertices by default.

In [133]:
setprecision(BigFloat, 2^7)
γs = sort([rand(BigFloat) for _ in 1:10])
lottery_γs = []
success_sum = []
K = [k for k in 1 : 1 : 20]
N = [n for n in 20 : 10 : 100]

graphs = [[barabasi_albert(N[j], K[i]) for j in 1:length(N)] for i ∈ 1:length(K)]

for γ ∈ γs
    success_matrix = zeros(length(P), length(N))
    for i ∈ 1:length(P)
        for j in 1:length(N)
            x₀, edge_index = construct_graph(graphs[i][j])
            iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
            coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
            success_matrix[i, j] = coloring_wl == coloring_gnn || maximum(coloring_wl) <= maximum(coloring_gnn)
            if coloring_wl != coloring_gnn
                println("P: $(P[i]), N: $(N[j])")
                println(coloring_wl)
                println(coloring_gnn)
                println(maximum(coloring_wl))
                println(maximum(coloring_gnn))
            end
        end
    end
    push!(success_sum, sum(success_matrix))
    if sum(success_matrix) == sum(ones(length(P), length(N)))
        println("lottery ticket gamma: $(γ)")
        push!(lottery_γs, γ) 
    end
end

ones_sum = sum(ones(length(P), length(N)))
plot(γs, [ones_sum for _ ∈ γs], label="WL", dpi=1000)
plot!(γs, success_sum, xaxis=:log, label="MPGNN")
scatter!(lottery_γs, [ones_sum for _ ∈ lottery_γs], 
        series_annotations = text.(lottery_γs, :left, 8, rotation=270), 
        label="Lottery γs")
ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPGNN, Barabasi Albert")
xlabel!("γ")
ylabel!("successful simulations")
savefig("results\\barabasi_albert.png")

P: 0.15, N: 80
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 9.0, 9.0, 15.0, 12.0, 16.0, 17.0, 9.0, 18.0, 12.0, 19.0, 9.0, 12.0, 20.0, 19.0, 21.0, 17.0, 20.0, 9.0, 9.0, 9.0, 12.0, 12.0, 19.0, 12.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 19.0, 22.0, 23.0, 9.0, 12.0, 12.0, 12.0, 9.0, 9.0, 12.0, 12.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0]
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 9.0, 9.0, 15.0, 12.0, 16.0, 17.0, 9.0, 18.0, 12.0, 19.0, 9.0, 12.0, 20.0, 19.0, 21.0, 22.0, 20.0, 9.0, 9.0, 9.0, 12.0, 12.0, 19.0, 12.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 19.0, 23.0, 24.0, 9.0, 12.0, 12.0, 12.0, 9.0, 9.0, 12.0, 12.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 12.0, 9.0, 9.0, 9.0, 9.0]
23.0
24.0
P: 0.35, N: 30
BigFloat[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13

"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\A Single Channel\\barabasi_albert.png"

In [54]:
K = [k for k in 1 : 1 : 20]
N = [n for n in 20 : 10 : 100]

graphs = [[barabasi_albert(N[j], K[i]) for j in 1:length(N)] for i ∈ 1:length(K)]

γs = sort([rand(BigFloat) for _ in 1:5])


precisions = [2^i for i ∈ 3:6]
success_rate = []
for precision ∈ precisions
    setprecision(BigFloat, precision)
    success_sum = []
    for γ ∈ γs
        success_matrix = zeros(length(P), length(N))
        for i ∈ 1:length(P)
            for j in 1:length(N)
                x₀, edge_index = construct_graph(graphs[i][j])
                iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
                coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
                success_matrix[i, j] = coloring_wl == coloring_gnn ||  maximum(coloring_wl) <= maximum(coloring_gnn)
                # if coloring_wl != coloring_gnn
                #     println("P: $(P[i]), N: $(N[j])")
                #     println(coloring_wl)
                #     println(BigFloat.(coloring_gnn))
                #     println(maximum(coloring_wl))
                #     println(maximum(coloring_gnn))
                # end
            end
        end
        push!(success_sum, sum(success_matrix))
        # if sum(success_matrix) == sum(ones(length(P), length(N)))
        #     println("lottery ticket gamma: $(γ)")
        #     push!(lottery_γs, γ) 
        # end
    end
    push!(success_rate, maximum(success_sum))
end
ones_sum = sum(ones(length(P), length(N)))
plot(precisions, [ones_sum for _ ∈ precisions], label="WL", xaxis=:log, dpi=1000)
plot!(precisions, success_rate, label="MPGNN")
scatter!(precisions, success_rate)
ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPGNN, Barabasi Albert")
xlabel!("γ")
ylabel!("successful simulations")
savefig("results\\bits_barabasi_albert.png")

"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\A Single Channel\\bits_barabasi_albert.png"

## Experimento: Big Erdos

In [134]:
graph = erdos_renyi(1000, 0.1)
x₀ = ones(Int, nv(graph))
edge_index = zeros(Int, 2, ne(graph))
for (i, edge) in zip(1:ne(graph), edges(graph))
    edge_index[1, i] = edge.src
    edge_index[2, i] = edge.dst
end

In [135]:
iters, coloring = WeisfeilerLehman(x₀, edge_index)
big_erdos_renyi_sum = sum(coloring)
println(iters)

5


In [127]:
# γs = vcat(sort([rand(0 : .1e-16 : 1e-8) for _ in 1:20]), sort([rand(1e-4 : .1e-16 : 1e-2) for _ in 1:20]))
γs = sort([rand(0 : .1e-16 : 0.5) for _ in 1:10])
simulations_sums = []
lottery_γs = []

for γ ∈ γs
    coloring_mpgnn = MPGNN(x₀, edge_index, γ, iters)
    gnn_sum = sum(coloring_mpgnn)
    push!(simulations_sums, gnn_sum)
    if big_erdos_renyi_sum == gnn_sum
        println(coloring == coloring_mpgnn)
        println("lottery ticket gamma: $(γ)")
        # comparar particiones verificar
        push!(lottery_γs, γ)
    end 
end

plot(γs, [big_erdos_renyi_sum for _ ∈ γs], label="WL")
plot!(γs, simulations_sums, xaxis=:log, label="MPGNN", dpi=1000)
scatter!(lottery_γs, [big_erdos_renyi_sum for _ ∈ lottery_γs], 
        series_annotations = text.(lottery_γs, :left, 8, rotation=270), 
        label="Lottery γs")
        
# ylims!(0, cora_sum)
title!("WL vs 1D-MPGNN, Big Erdős Renyi")
xlabel!("γ")
ylabel!("successful simulations")
savefig("results\\big_erdos_renyi.png")

true
lottery ticket gamma: 0.061455415818785046
true
lottery ticket gamma: 0.11366281655891275
true
lottery ticket gamma: 0.12661270004210862
true
lottery ticket gamma: 0.13418572213638513
true
lottery ticket gamma: 0.16008127840925448
true
lottery ticket gamma: 0.1975446688576497
true
lottery ticket gamma: 0.20468800199343384
true
lottery ticket gamma: 0.22808877796473764
true
lottery ticket gamma: 0.28263736332406436
true
lottery ticket gamma: 0.42751465553444645


"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\A Single Channel\\big_erdos_renyi.png"

## Experimento Cora

In [136]:
dataset_directory = "data/"
dataset = pyg.datasets.Planetoid(dataset_directory, "Cora")
x₀ = ones(2708)
data = dataset[0]

PyObject Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [137]:
iters, coloring = WeisfeilerLehman(x₀, data.edge_index.numpy() .+ 1)
cora_sum = sum(coloring)
iters

6

In [130]:
include("src/weisfeiler_lehman.jl")
include("src/mpgnn.jl")
include("src/utils.jl")

γs = [1e0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
γs = sort([rand(0 : .1e-15 : 1.) for _ in 1:10])
# γs = [0.0529543998481467, 0.1524684965510319, 0.1968312871873878,
#     0.2394649845179258, 0.2636596628241623, 0.3042315663142059,
#     0.38465258902648397, 0.4435650207271238, 0.4589774009623747]
results = []
lottery_γs = []
buenos = 0
for γ ∈ γs
    coloring_mpgnn = MPGNN(x₀, data.edge_index.numpy() .+ 1, γ, iters)
    gnn_sum = sum(coloring_mpgnn)
    # push!(results, gnn_sum)
    if cora_sum == gnn_sum
        println(coloring == coloring_mpgnn)
        buenos += 1
        push!(results, 1)
        println("lottery gamma: $(γ)")
        push!(lottery_γs, γ)
    else 
        push!(results, 0)
    end 
end

println(buenos)

plot(γs, [1 for _ ∈ γs], label="WL", dpi=1000)
plot!(γs, results, xaxis=:log, label="MPGNN")
scatter!(lottery_γs, [1 for _ ∈ lottery_γs], 
        series_annotations = text.(lottery_γs, :left, 8, rotation=270), 
        label="Lottery γs")
ylims!(0, 1)
title!("WL vs 1D-MPGNN, Cora")
xlabel!("γ")
ylabel!("successful simulations")
savefig("results\\cora.png")

# true
# lottery gamma: 0.0529543998481467
# true
# lottery gamma: 0.1524684965510319
# true
# lottery gamma: 0.1968312871873878
# true
# lottery gamma: 0.2394649845179258
# true
# lottery gamma: 0.2636596628241623
# true
# lottery gamma: 0.3042315663142059
# true
# lottery gamma: 0.3073965762757351
# true
# lottery gamma: 0.38465258902648397
# true
# lottery gamma: 0.4435650207271238
# true
# lottery gamma: 0.4589774009623747
# 10

0


"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\A Single Channel\\cora.png"